# Cell-penetrating peptides (CPP) prediction

This notebook focus on linear peptides with all natural amino acids.

In [1]:
import pandas as pd
import numpy as np

train = False

# CPP924 dataset

In [2]:
df_train_cpp924 = pd.read_csv('data/CPP924/train.csv')
df_test_cpp924 = pd.read_csv('data/CPP924/test.csv')

In [3]:
y_train = df_train_cpp924.is_cpp.values
y_test = df_test_cpp924.is_cpp.values

In [4]:
from sklearn.metrics import accuracy_score, recall_score, matthews_corrcoef, roc_auc_score

def get_metrics(y_hat, y_test):
    acc = accuracy_score(y_test, y_hat)
    sn = recall_score(y_test, y_hat)
    sp = recall_score(y_test, y_hat, pos_label=0)
    mcc = matthews_corrcoef(y_test, y_hat)
    auroc = roc_auc_score(y_test, y_hat)

    print(f'Acc(%) \t Sn(%) \t Sp(%) \t MCC \t AUROC')
    print(f'{acc*100:.2f}\t{sn*100:.2f}\t{sp*100:.2f}\t{mcc:.3f}\t{auroc:.3f}')


## Feature processing

### Fingerprints

In [5]:
import warnings
from rdkit import Chem, rdBase, DataStructs
from rdkit.Chem import AllChem
from typing import List

rdBase.DisableLog('rdApp.error')
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

def fingerprints_from_smiles(smiles: List, size=2048):
    """
        Create ECFP fingerprints of smiles, with validity check
    """
    fps = []
    valid_mask = []
    for i, smile in enumerate(smiles):
        mol = Chem.MolFromSmiles(smile)
        valid_mask.append(int(mol is not None))
        fp = fingerprints_from_mol(mol, size=size) if mol else np.zeros((1, size))
        fps.append(fp)

    fps = np.concatenate(fps, axis=0)
    return fps, valid_mask


def fingerprints_from_mol(molecule, radius=3, size=2048, hashed=False):
    """
        Create ECFP fingerprint of a molecule
    """
    if hashed:
        fp_bits = AllChem.GetHashedMorganFingerprint(molecule, radius, nBits=size)
    else:
        fp_bits = AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=size)
    fp_np = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp_bits, fp_np)
    return fp_np.reshape(1, -1)

In [6]:
# train = True
if train:
    X_train = fingerprints_from_smiles(df_train_cpp924.smi)[0]
    X_test = fingerprints_from_smiles(df_test_cpp924.smi)[0]

    np.save('data/CPP924/X_train_fps.npy', X_train)
    np.save('data/CPP924/X_test_fps.npy', X_test)
else:
    X_train = np.load('data/CPP924/X_train_fps.npy')
    X_test = np.load('data/CPP924/X_test_fps.npy')

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((733, 2048), (733,), (183, 2048), (183,))

### ESM-2 features

In [7]:
import torch
import esm

def load_esm_model():
    model, alphabet = esm.pretrained.esm2_t33_650M_UR50D() # E=1280 90.71	93.88	87.06	0.814	0.905
    batch_converter = alphabet.get_batch_converter()
    model.eval()  # disables dropout for deterministic results
    return model, batch_converter, alphabet

def get_esm_seq_representation(aa_seqs, batch_converter, model, alphabet, token_embd="mean"):

    data = [(f"seq{id}", seq) for id, seq in enumerate(aa_seqs)]
    _, _, batch_tokens = batch_converter(data)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[-1], return_contacts=True)
    token_representations = results["representations"][-1]

    if token_embd == "mean":
        sequence_representations = []
        for i, tokens_len in enumerate(batch_lens):
            sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0)) # Take mean of non-pad tokens
        
        return torch.stack(sequence_representations).numpy()
    else:
        return token_representations.numpy()

In [8]:
# train = True
train = False
if train:
    model, batch_converter, alphabet = load_esm_model()
    X_train = get_esm_seq_representation(df_train_cpp924.aa_seq, batch_converter, model, alphabet)
    X_test = get_esm_seq_representation(df_test_cpp924.aa_seq, batch_converter, model, alphabet)

    np.save('data/CPP924/X_train_esm2.npy', X_train)
    np.save('data/CPP924/X_test_esm2.npy', X_test)
else:
    X_train = np.load('data/CPP924/X_train_esm2.npy')
    X_test = np.load('data/CPP924/X_test_esm2.npy')

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((733, 1280), (733,), (183, 1280), (183,))

### BERT features

In [9]:
import torch
import numpy as np

from utils.utils import parse_config, load_model, get_metrics
from models.bert import BERT
from datasets.tokenizer import SmilesTokenizer, AATokenizer

def load_bert_model(ckpt, config, device='cuda', model_type='smi_bert'):
    if model_type == 'smi_bert':
        tokenizer = SmilesTokenizer(max_len=config.data.max_len)
    elif model_type == 'aa_bert':
        tokenizer = AATokenizer(max_len=config.data.max_len)
    else:
        raise ValueError(f'Invalid model_type: {model_type}')

    model = BERT(tokenizer, **config.model)
    model = load_model(model, ckpt, device)
    model.eval()
    return model, device

def get_bert_embd(encoder, inputs, device='cuda', cls_token=False):
    tokens = encoder.tokenize_inputs(inputs).to(device)
    embd = encoder.embed(tokens)
    if cls_token:
        del tokens
        return embd[:, 0, :].squeeze()
    else:
        batch_lens = (tokens != encoder.tokenizer.pad_token_id).sum(1)
        reps = []
        for i, tokens_len in enumerate(batch_lens):
            reps.append(embd[i, 1 : tokens_len - 1].mean(0))
        del tokens, batch_lens, embd
        return torch.stack(reps)

def encode_with_bert(list, model, device='cuda', cls_token=False):
    with torch.no_grad():
        output= get_bert_embd(model, list, device=device, cls_token=cls_token)
        embd = output.cpu().numpy()
    return embd

#### SMILES BERT

In [10]:
# train = True
train = False
if train:
    ckpt='results/train_smi_bert_tune/model_13_0.003.pt'  # 90.71	93.88	87.06	0.814	0.905
    config_file='configs/train_smi_bert_tune.yaml'
    config = parse_config(config_file)
    model, device = load_bert_model(ckpt=ckpt, config=config)

    X_train = encode_with_bert(df_train_cpp924.smi, model)
    X_test = encode_with_bert(df_test_cpp924.smi, model)

    np.save('data/CPP924/X_train_smi_bert.npy', X_train)
    np.save('data/CPP924/X_test_smi_bert.npy', X_test)
else:
    X_train = np.load('data/CPP924/X_train_smi_bert.npy')
    X_test = np.load('data/CPP924/X_test_smi_bert.npy')

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((733, 512), (733,), (183, 512), (183,))

#### AA BERT

In [11]:
# train = True
train = False
if train:
    ckpt='results/train_aa_bert/model_12_2.523.pt'  # 94.54	97.96	90.59	0.892	0.943
    config_file='configs/train_aa_bert_test.yaml'
    config = parse_config(config_file)
    model, device = load_bert_model(ckpt=ckpt, config=config, model_type='aa_bert')

    X_train = encode_with_bert(df_train_cpp924.aa_seq, model)
    X_test = encode_with_bert(df_test_cpp924.aa_seq, model)

    np.save('data/CPP924/X_train_aa_bert.npy', X_train)
    np.save('data/CPP924/X_test_aa_bert.npy', X_test)
else:
    X_train = np.load('data/CPP924/X_train_aa_bert.npy')
    X_test = np.load('data/CPP924/X_test_aa_bert.npy')

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((733, 256), (733,), (183, 256), (183,))

### MolClip features

In [12]:
import yaml
import torch
import numpy as np
from models.molclip import MolCLIP
from utils.utils import load_model, parse_config

def load_molclip_model(ckpt, config, device='cuda'):
    model = MolCLIP(device=device, config=config.model)
    model = load_model(model, ckpt, device)
    model.eval()
    return model

def get_molclip_embd(model, smiles, aa_seq):
    with torch.no_grad():
        smi_output, _ = model.get_smi_embd(smiles)
        smi_feat = model.smi_proj(smi_output)
        aa_output, _ = model.get_aa_embd(aa_seq)
        aa_feat = model.aa_proj(aa_output)
        feat = torch.cat([smi_feat, aa_feat], dim=1)
        embd = feat.cpu().numpy()
    return embd

In [13]:
# train = True
train = False
if train:
    config = './configs/train_molclip_test.yaml'
    # ckpt='results/train_molclip_test/model_0_0.000.pt'  # 88.52	87.76	89.41	0.770	0.886
    ckpt = 'results/train_molclip/model_1_2.618.pt' # 87.98	89.80	85.88	0.758	0.878
    model = load_molclip_model(ckpt=ckpt, config=parse_config(config))
    
    X_train = get_molclip_embd(model, df_train_cpp924.smi, df_train_cpp924.aa_seq)
    X_test = get_molclip_embd(model, df_test_cpp924.smi, df_test_cpp924.aa_seq)
    
    np.save('data/CPP924/X_train_mol_clip.npy', X_train)
    np.save('data/CPP924/X_test_mol_clip.npy', X_test)
else:
    X_train = np.load('data/CPP924/X_train_mol_clip.npy')
    X_test = np.load('data/CPP924/X_test_mol_clip.npy')

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((733, 512), (733,), (183, 512), (183,))

### BART features

In [18]:
import torch
import numpy as np

from utils.utils import parse_config, load_model, get_metrics
from models.pbart import PepBART

def load_pep_bart_model(ckpt, config, device='cuda'):

    model = PepBART(device=device, config=config.model).to(device)
    model = load_model(model, ckpt, device)
    model.eval()
    return model, device

def get_pep_bart_embd(model, inputs, device='cuda'):
    tokens = model.aa_encoder.tokenize_inputs(inputs).to(device)
    embd = model.aa_encoder.embed(tokens)
    batch_lens = (tokens != model.aa_encoder.tokenizer.pad_token_id).sum(1)
    reps = []
    for i, tokens_len in enumerate(batch_lens):
        reps.append(embd[i, 1 : tokens_len - 1].mean(0))
    del tokens, batch_lens, embd
    return torch.stack(reps)

def encode_with_pep_bart(list, model, device='cuda'):
    with torch.no_grad():
        output= get_pep_bart_embd(model, list, device=device)
        embd = output.cpu().numpy()
    return embd

In [19]:
train = True
# train = False
if train:
    ckpt='results/train_pep_bart/model_1_1.246.pt'
    config_file='configs/train_pep_bart_test.yaml'
    config = parse_config(config_file)
    model, device = load_pep_bart_model(ckpt=ckpt, config=config)

    X_train = encode_with_pep_bart(df_train_cpp924.aa_seq, model)
    X_test = encode_with_pep_bart(df_test_cpp924.aa_seq, model)

    np.save('data/CPP924/X_train_pep_bart.npy', X_train)
    np.save('data/CPP924/X_test_pep_bart.npy', X_test)
else:
    X_train = np.load('data/CPP924/X_train_pep_bart.npy')
    X_test = np.load('data/CPP924/X_test_pep_bart.npy')

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((733, 256), (733,), (183, 256), (183,))

## Classification

In [26]:
# features = ['fps', ]
# features = ['esm2', ] 
# features = ['smi_bert', ]
# features = ['aa_bert', ] 
# features = ['molclip', ] 
features = ['pep_bart', ] 
# features = ['esm2', 'fps', ]  
# features = ['esm2', 'smi_bert', ]
# features = ['esm2', 'fps', 'smi_bert']
# features = ['smi_bert', 'aa_bert', ]

X_train_features = []
X_test_features = []
for feat in features:
    if feat == 'esm2':
        X_train = np.load('data/CPP924/X_train_esm2.npy')
        X_test = np.load('data/CPP924/X_test_esm2.npy')
        
        X_train_features.append(X_train)
        X_test_features.append(X_test)
    elif feat == 'fps':
        X_train = np.load('data/CPP924/X_train_fps.npy')
        X_test = np.load('data/CPP924/X_test_fps.npy')

        X_train_features.append(X_train)
        X_test_features.append(X_test)
    elif feat == 'smi_bert':
        X_train = np.load('data/CPP924/X_train_smi_bert.npy')
        X_test = np.load('data/CPP924/X_test_smi_bert.npy')

        X_train_features.append(X_train)
        X_test_features.append(X_test)
    elif feat == 'aa_bert':
        X_train = np.load('data/CPP924/X_train_aa_bert.npy')
        X_test = np.load('data/CPP924/X_test_aa_bert.npy')

        X_train_features.append(X_train)
        X_test_features.append(X_test)
    elif feat == 'molclip':
        X_train = np.load('data/CPP924/X_train_mol_clip.npy')
        X_test = np.load('data/CPP924/X_test_mol_clip.npy')

        X_train_features.append(X_train)
        X_test_features.append(X_test)
    elif feat == 'pep_bart':
        X_train = np.load('data/CPP924/X_train_pep_bart.npy')
        X_test = np.load('data/CPP924/X_test_pep_bart.npy')

        X_train_features.append(X_train)
        X_test_features.append(X_test)
    else:
        raise ValueError(f'Feature {feat} not supported')

X_train = np.concatenate(X_train_features, axis=1)
X_test = np.concatenate(X_test_features, axis=1)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((733, 256), (733,), (183, 256), (183,))

### Random Forest

In [27]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

get_metrics(y_hat, y_test)

Acc(%) 	 Sn(%) 	 Sp(%) 	 MCC 	 AUROC
91.80	93.88	89.41	0.835	0.916


(0.9180327868852459,
 0.9387755102040817,
 0.8941176470588236,
 0.8353032703034405,
 0.9164465786314526)